[periodo-reconciler/API.md at master · periodo/periodo-reconciler](https://github.com/periodo/periodo-reconciler/blob/master/API.md)

In [1]:
import requests
import json
import uuid
from collections import OrderedDict
import urllib.parse

In [2]:
from periodo_reconciler import (
    RProperty,
    RQuery,
    PeriodoReconciler
)

In [3]:
p_recon = PeriodoReconciler(host='localhost:8142')
p_recon

PeriodoReconciler(host="localhost:8142", protocol="http")

In [4]:
set(p_recon.describe().keys())

{'defaultTypes',
 'identifierSpace',
 'name',
 'preview',
 'schemaSpace',
 'suggest',
 'view'}

# Reconcile period names 


In [5]:
q = """
{
  "basic-query": {
    "query": "北宋"
  },
  "limited-query": {
    "query": "bronze age",
    "limit": 1
  },
  "additional-properties-query": {
    "query": "Ранньоримський",
    "properties": [
      {
        "p": "location",
        "v": "Ukraine"
      },
      {
        "p": "start",
        "v": "200"
      },
      {
        "p": "stop",
        "v": "600"
      }
    ]
  }
}

""".strip()

json.loads(q)

{'additional-properties-query': {'properties': [{'p': 'location',
    'v': 'Ukraine'},
   {'p': 'start', 'v': '200'},
   {'p': 'stop', 'v': '600'}],
  'query': 'Ранньоримський'},
 'basic-query': {'query': '北宋'},
 'limited-query': {'limit': 1, 'query': 'bronze age'}}

In [6]:
r = requests.get('http://localhost:8142', params={'queries':q})
r.json()

{'additional-properties-query': {'result': [{'id': 'http://n2t.net/ark:/99152/p06v8w4dbcf',
    'match': True,
    'name': 'Ранньоримський період [Ukraine, Ukraine: -0049 to 0175]',
    'score': 0,
    'type': [{'id': 'http://www.w3.org/2004/02/skos/core#Concept',
      'name': 'Period definition'}]}]},
 'basic-query': {'result': [{'id': 'http://n2t.net/ark:/99152/p0fp7wvjvn8',
    'match': True,
    'name': 'Northern Song [China, China: 0960 to 1127]',
    'score': 0,
    'type': [{'id': 'http://www.w3.org/2004/02/skos/core#Concept',
      'name': 'Period definition'}]}]},
 'limited-query': {'result': [{'id': 'http://n2t.net/ark:/99152/p0z3skmnss7',
    'match': False,
    'name': 'Bronze [Palestine, Israel, Jordan: -3299 to -1199]',
    'score': 0,
    'type': [{'id': 'http://www.w3.org/2004/02/skos/core#Concept',
      'name': 'Period definition'}]}]}}

In [7]:
queries = [
    RQuery("北宋", label="basic-query"),
    RQuery("bronze age", label="limited-query", limit=1),
    RQuery("Ранньоримський", label="additional-properties-query", properties=[
        RProperty('location', 'Ukraine'),
        RProperty('start', 200),
        RProperty('end', 600)
    ])
]

queries

[RQuery("\u5317\u5b8b", label="basic-query"),
 RQuery("bronze age", label="limited-query", limit=1),
 RQuery("\u0420\u0430\u043d\u043d\u044c\u043e\u0440\u0438\u043c\u0441\u044c\u043a\u0438\u0439", label="additional-properties-query", properties=[RProperty("location", "Ukraine"),
 RProperty("start", 200),
 RProperty("end", 600)])]

In [8]:
r = p_recon.reconcile(queries, method='post')

In [9]:
result = p_recon.suggest_properties()
result

[{'id': 'location', 'name': 'Spatial coverage'},
 {'id': 'start', 'name': 'Year or start year'},
 {'id': 'stop', 'name': 'End year'}]

In [10]:
type(result)

list

In [11]:
p_recon.suggest_entities('bashkekohore')

[{'id': 'http://n2t.net/ark:/99152/p06v8w4nfqg',
  'match': False,
  'name': 'Bashkëkohore [Albania, Albania: 1944 to 2000]',
  'score': 0,
  'type': [{'id': 'http://www.w3.org/2004/02/skos/core#Concept',
    'name': 'Period definition'}]},
 {'id': 'http://n2t.net/ark:/99152/p0qhb662s6j',
  'match': False,
  'name': 'Bashkëkohore [Albania, Albania: 1944 to 2000]',
  'score': 0,
  'type': [{'id': 'http://www.w3.org/2004/02/skos/core#Concept',
    'name': 'Period definition'}]}]

In [12]:
p_recon.preview_period('http://n2t.net/ark:/99152/p0fp7wvjvn8')

b'<!doctype html><html><body bgcolor="white">\n    <style>\n      dl {\n        font-family: sans-serif;\n        font-size: 13px !important;\n        margin: 0.5em !important;\n      }\n      dh { color: #93a1a1 }\n      dd {\n        margin-bottom: 1em !important;\n        margin-left: 1em !important;\n      }\n      p { margin: 0.5em 0; font-size: 13px !important; }\n      a { color: #4272db; text-decoration: none }\n    </style>\n    <dl>\n      <dh>Period</dh>\n      <dd>\n        <p><a target="_blank" href="http://n2t.net/ark:/99152/p0fp7wvjvn8">Northern Song</a></p>\n        <p><i>Beisong, \xe5\x8c\x97\xe5\xae\x8b, \xe5\x8c\x97\xe5\xae\x8b</i></p>\n      </dd>\n\n      <dh>Source</dh>\n      <dd>\n        <p>\n          <a target="_blank"\n             href="http://n2t.net/ark:/99152/p0fp7wv">Merrick Lex Berman. CHGIS: Major Chinese Periods Chronology. 2015.</a>\n        </p>\n        <p>\n          <i>contains 52 period definitions</i>\n        </p>\n      </dd>\n\n      <dh>Ea

In [13]:
r = p_recon.preview_period('http://n2t.net/ark:/99152/p0fp7wvjvn8', flyout=True)

In [14]:
RProperty('location', 'Ukraine')

RProperty("location", "Ukraine")

In [15]:
RQuery("bronze age")

RQuery("bronze age", label="e3db9211-52f2-4f49-aa82-951aa0998bce")

In [16]:
RQuery("bronze age", label="limited-query", limit=1)

RQuery("bronze age", label="limited-query", limit=1)

In [17]:
RQuery("bronze age", label="limited-query")

RQuery("bronze age", label="limited-query")

In [18]:
RQuery("Ранньоримський", label="additional-properties-query", properties=[
        RProperty('location', 'Ukraine'),
        RProperty('start', 200),
        RProperty('end', 600)
    ])

RQuery("\u0420\u0430\u043d\u043d\u044c\u043e\u0440\u0438\u043c\u0441\u044c\u043a\u0438\u0439", label="additional-properties-query", properties=[RProperty("location", "Ukraine"),
RProperty("start", 200),
RProperty("end", 600)])

In [19]:
q = RQuery("Ранньоримський", label="additional-properties-query", properties=[
        RProperty('location', 'Ukraine'),
        RProperty('start', 200),
        RProperty('end', 600)
    ])

In [20]:
repr(q)

'RQuery("\\u0420\\u0430\\u043d\\u043d\\u044c\\u043e\\u0440\\u0438\\u043c\\u0441\\u044c\\u043a\\u0438\\u0439", label="additional-properties-query", properties=[RProperty("location", "Ukraine"),\nRProperty("start", 200),\nRProperty("end", 600)])'

# PKAP

In [21]:
queries = [

    RQuery("Late Roman", label="PKAP Late Roman", properties=[
        RProperty('location', 'Cyprus'),
        RProperty('start', 300),
        RProperty('end', 749)
    ]), 
    RQuery("Roman, Late", label="PKAP Roman, Late",  properties=[
        RProperty('location', 'Cyprus'),
        RProperty('start', 300),
        RProperty('end', 749)
    ]),    

]

queries

[RQuery("Late Roman", label="PKAP Late Roman", properties=[RProperty("location", "Cyprus"),
 RProperty("start", 300),
 RProperty("end", 749)]),
 RQuery("Roman, Late", label="PKAP Roman, Late", properties=[RProperty("location", "Cyprus"),
 RProperty("start", 300),
 RProperty("end", 749)])]

In [22]:
r = p_recon.reconcile(queries, method='post')
r

{'PKAP Late Roman': {'result': [{'id': 'http://n2t.net/ark:/99152/p077pzfxk2d',
    'match': False,
    'name': 'Late Roman [Roman Empire, Roman Empire: 0235 to 0410]',
    'score': 0,
    'type': [{'id': 'http://www.w3.org/2004/02/skos/core#Concept',
      'name': 'Period definition'}]},
   {'id': 'http://n2t.net/ark:/99152/p08m57h5vpv',
    'match': False,
    'name': 'Late Roman [Western Roman Empire, Italy, France, Belgium, Luxembourg, Spain, Portugal, Switzerland, Germany, England, Morocco, Algeria, Tunisia, Libya, Austria, Hungary, Slovenia, Croatia, Bosnia and Herzegovina: 0293 to 0476]',
    'score': 0,
    'type': [{'id': 'http://www.w3.org/2004/02/skos/core#Concept',
      'name': 'Period definition'}]},
   {'id': 'http://n2t.net/ark:/99152/p0dg76fbqff',
    'match': False,
    'name': 'Late Roman [Cyprus, Cyprus: 0300 to 0749]',
    'score': 0,
    'type': [{'id': 'http://www.w3.org/2004/02/skos/core#Concept',
      'name': 'Period definition'}]},
   {'id': 'http://n2t.net/a

Conclusion: "Roman, Late" brings up no results whereas "Late Roman" brings up many results.